In [ ]:
# 1. Star
# 2. Folk
# 3. Watching
# 4. Commits
# 5. Issue
# 6. PL 
# 7. Pull Request
# 8. Actions 
# 9. Docs 
# 10. Contributor
# 11. Code

In [4]:
import datetime
import pandas as pd
import os
import json
from collections import defaultdict

In [ ]:
# the metrics calculated will be saved in '../metrics.csv'

In [5]:
repos_data_path = "/Users/zhangyujin/Desktop/repos-data"
metrics_path = "../metrics.csv"
metrics = pd.read_csv(metrics_path)

In [ ]:
# 01 repo_size

In [23]:
metrics['stars'] = None
metrics['forks'] = None
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    repo_path = f"{repos_data_path}/{owner}_{repo}"
    if os.path.exists(repo_path):
        repo_info_path = f"{repos_data_path}/{owner}_{repo}/repo.json"
        if os.path.exists(repo_info_path):
            with open(repo_info_path, 'r') as f:
                repo_info = json.load(f)
            stars = repo_info.get('stargazers_count', 0)
            forks = repo_info.get('forks_count', 0)
            metrics.at[index, 'stars'] = stars
            metrics.at[index, 'forks'] = forks

In [24]:
metrics.to_csv(metrics_path, index=False)

In [ ]:
# 02 longitudinal(Recent 500 issues/pulls/commits)

In [6]:
metrics['issue_closed_rate'] = None
metrics['issue_time_to_close'] = None
metrics['issue_time_to_response'] = None
metrics['issue_monthly'] = None
metrics['issue_participants'] = None

In [12]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        closed_issues_count = 0
        labeled_issues_count = 0
        commented_issues_count = 0
        total_comments = 0
        total_time_to_comment = 0
        total_time_to_close = 0
        monthly_issues_count = defaultdict(int)
        participants = set()
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:

            labeled_issues_count += 1 if issue['labels'] else 0
            created_at = datetime.datetime.strptime(issue['created_at'], '%Y-%m-%dT%H:%M:%SZ')
            monthly_issues_count[(created_at.year, created_at.month)] += 1

            if issue['state'] == 'closed':
                closed_issues_count += 1
                closed_at = datetime.datetime.strptime(issue['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
                total_time_to_close += (closed_at - created_at).total_seconds()

            comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{issue['number']}.json"
            
            if os.path.exists(comments_path):
                
                with open(comments_path, 'r') as f:
                    comments = json.load(f)
                    
                total_comments += len(comments)
                first_comment_time = datetime.datetime.strptime(comments[0]['created_at'], '%Y-%m-%dT%H:%M:%SZ')
                time_to_comment = (first_comment_time - created_at).total_seconds()
                total_time_to_comment += time_to_comment
                commented_issues_count += 1
                
                for comment in comments:
                    participant = comment['user']['login']
                    participants.add(participant)
        
    metrics.at[index, 'issue_closed_rate'] = f"{(closed_issues_count / len(issues)):.1f}" if issues else "N/A"
    metrics.at[index, 'issue_time_to_close'] = f"{(total_time_to_close / closed_issues_count / (24 * 3600)):.1f}" if closed_issues_count > 0 else "N/A"
    metrics.at[index, 'issue_time_to_response'] = f"{(total_time_to_comment / commented_issues_count / (24 * 3600)):.1f}" if commented_issues_count > 0 else "N/A"
    metrics.at[index, 'issue_monthly'] = f"{(sum(monthly_issues_count.values()) / len(monthly_issues_count)):.1f}" if len(monthly_issues_count) > 0 else "N/A"
    metrics.at[index, 'issue_participants'] = f"{len(participants):d}"